In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [47]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
        

In [48]:
net = Net()

In [49]:
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [50]:
params = list(net.parameters())

In [51]:
params[0].size()

torch.Size([6, 1, 3, 3])

In [52]:
len(params)

10

In [53]:
input = torch.randn(1, 1, 32, 32)
out = net(input)

In [54]:
out

tensor([[ 0.0615,  0.0531, -0.0450, -0.0332,  0.0924,  0.0230,  0.0791,  0.1099,
          0.0378, -0.0290]], grad_fn=<AddmmBackward>)

In [55]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [56]:
output = net(input)

In [57]:
target = torch.randn(10)  # a dummy target, for example

In [58]:
target

tensor([ 0.5304, -0.7438, -0.3570,  1.1586, -0.2523,  0.5106, -0.8306, -0.2555,
         1.9090,  0.3621])

In [59]:
target = target.view(1, -1)  # make it the same shape as output

In [60]:
target

tensor([[ 0.5304, -0.7438, -0.3570,  1.1586, -0.2523,  0.5106, -0.8306, -0.2555,
          1.9090,  0.3621]])

In [61]:
criterion = nn.MSELoss()
loss = criterion(output, target)
loss

tensor(0.7344, grad_fn=<MseLossBackward>)

In [62]:
net.zero_grad()

In [63]:
net.conv1.bias.grad

tensor([0., 0., 0., 0., 0., 0.])

In [64]:
loss.backward()

In [65]:
net.conv1.bias.grad

tensor([ 0.0137,  0.0022, -0.0043, -0.0047,  0.0112, -0.0116])

In [38]:
loss.grad_fn

In [39]:
loss.grad_fn.next_functions[0][0]

In [40]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

In [66]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [67]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update